In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

Read the data and split them to in time and out of time

In [2]:
inTime = pd.read_csv('finalVariable_inTimeData3.csv')
outTime = pd.read_csv('finalVariable_outTimeData3.csv')

outTime = outTime.iloc[:,1:]
inTime = inTime.iloc[:,1:]

inTimeX = inTime.drop('FraudLabel', axis=1)
inTimeY = inTime.loc[:,'FraudLabel']
inTimeX.head(5)
#inTimeY.head(5)

,address-zip5-homephone_count30_date,address-zip5_0_count_address-zip5_30_count_Ave,address-zip5_0_count_address-zip5_7_count_Ave,address-zip5_count0_date,address-zip5_count180_date,address-zip5_count1_date,address-zip5_count30_date,address-zip5_count3_date,address-zip5_count7_date,address-zip5_count90_date,...,address_count1_date,address_count30_date,address_count3_date,address_count7_date,address_count90_date,name-dob_count180_date,name-dob_count30_date,name-dob_count90_date,ssn-dob_count180_date,ssn-dob_count30_date
0,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
inTime[inTime['FraudLabel'] == 1].shape

(11542, 26)

In [4]:
inTime.shape

(800542, 26)

In [5]:
inTime[inTime['FraudLabel'] == 1].shape

(11542, 26)

In [6]:
inTime[inTime['FraudLabel'] == 0].shape

(789000, 26)

In [7]:
inTime.shape

(800542, 26)

In [8]:
outTime.shape

(166493, 26)

split in time into train and test

In [9]:
from sklearn.model_selection import train_test_split
Y = inTimeY
X = inTimeX
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, )


x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

fit the adaboosting model

In [20]:
'''%%time

for depth in [1,2,3,4,5,6,7,8,9,10]:
    for estimator in [100]:
        for leaf in [1,4,8,12,16,20,30]:
            
            training_FDR = list()
            testing_FDR = list()
            OOT_FDR = list()
            
            for i in list(range(1,5)):

                regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=depth, min_samples_leaf = leaf),
                                        n_estimators=estimator,loss='linear')

                regr.fit(x_train, y_train)

                #make predictions and caculate training FDR
                PredictTrainY = regr.predict(x_train)

                PredictTrainY = pd.Series(PredictTrainY)
                PredictTrainY.head(5)

                temp = pd.DataFrame(columns = ["PredictTrainY","RealY"])
                temp["PredictTrainY"] = PredictTrainY
                temp["RealY"] = y_train
                temp.sort_values(by = "PredictTrainY", ascending = False, inplace = True)

                ThreePercent = int(len(x_train) *0.03)

                FDR = temp["RealY"][:ThreePercent].sum() / y_train.sum()
                training_FDR.append(FDR)
                #print('training FDR' ,FDR)

                #make predictions and caculate test FDR
                PredictTestY = regr.predict(x_test)
                PredictTestY = pd.Series(PredictTestY)
                PredictTestY

                temp = pd.DataFrame(columns = ["PredictTestY","RealY"])
                temp["PredictTestY"] = PredictTestY
                temp["RealY"] = y_test
                temp.sort_values(by = "PredictTestY", ascending = False, inplace = True)

                ThreePercent = int(len(x_test) *0.03)
                ThreePercent

                FDR = temp["RealY"][:ThreePercent].sum() / y_test.sum()
                testing_FDR.append(FDR)
                #print('testing FDR' ,FDR)

                #make predictions and caculate OOT FDR
                outTimeX = outTime.drop('FraudLabel', axis=1)
                outTimeY = outTime.loc[:,'FraudLabel']

                PredictedY = regr.predict(outTimeX)
                PredictedY = pd.Series(PredictedY)

                temp = pd.DataFrame(columns = ["PredictedY","RealY"])
                temp["PredictedY"] = PredictedY
                temp["RealY"] = outTimeY
                temp.sort_values(by = "PredictedY", ascending = False, inplace = True)

                ThreePercent = int(len(outTimeX) *0.03)
                ThreePercent

                FDR = temp["RealY"][:ThreePercent].sum() / outTimeY.sum()
                OOT_FDR.append(FDR)
                #print('OOT FDR' ,FDR)

            print('max_depth', depth)
            print('n_estimators', estimator)
            print('min_samples_leaf',leaf)
            print('training FDR', np.max(training_FDR))
            print('testing FDR', np.max(testing_FDR))
            print('OOT FDR', np.max(OOT_FDR))
            print('-----------------------------------------')'''

max_depth 1
n_estimators 100
min_samples_leaf 1
training FDR 0.3551877096180894
testing FDR 0.34841235319704217
OOT FDR 0.3160100586756077
-----------------------------------------
max_depth 1
n_estimators 100
min_samples_leaf 4
training FDR 0.3551877096180894
testing FDR 0.34841235319704217
OOT FDR 0.3160100586756077
-----------------------------------------
max_depth 1
n_estimators 100
min_samples_leaf 8
training FDR 0.3551877096180894
testing FDR 0.34841235319704217
OOT FDR 0.3160100586756077
-----------------------------------------
max_depth 1
n_estimators 100
min_samples_leaf 12
training FDR 0.3551877096180894
testing FDR 0.34841235319704217
OOT FDR 0.3160100586756077
-----------------------------------------
max_depth 1
n_estimators 100
min_samples_leaf 16
training FDR 0.3551877096180894
testing FDR 0.34841235319704217
OOT FDR 0.3160100586756077
-----------------------------------------
max_depth 1
n_estimators 100
min_samples_leaf 20
training FDR 0.3551877096180894
testing FDR 

max_depth 7
n_estimators 100
min_samples_leaf 16
training FDR 0.5393270583144001
testing FDR 0.5237059591126577
OOT FDR 0.508382229673093
-----------------------------------------
max_depth 7
n_estimators 100
min_samples_leaf 20
training FDR 0.5385697284431462
testing FDR 0.5245759025663332
OOT FDR 0.5054484492875104
-----------------------------------------
max_depth 7
n_estimators 100
min_samples_leaf 30
training FDR 0.5379205885535
testing FDR 0.5219660722053067
OOT FDR 0.5058675607711651
-----------------------------------------
max_depth 8
n_estimators 100
min_samples_leaf 1
training FDR 0.5387861084063615
testing FDR 0.5215311004784688
OOT FDR 0.5041911148365466
-----------------------------------------
max_depth 8
n_estimators 100
min_samples_leaf 4
training FDR 0.5433300876338851
testing FDR 0.5245759025663332
OOT FDR 0.5075440067057837
-----------------------------------------
max_depth 8
n_estimators 100
min_samples_leaf 8
training FDR 0.540084388185654
testing FDR 0.52414093

KeyboardInterrupt: 

In [10]:


for depth in [8]:
    for estimator in [100]:
        for leaf in [16]:
            
            for FDR_Rate in [0.01, 0.03,0.05, 0.2]:
            
                training_FDR = list()
                testing_FDR = list()
                OOT_FDR = list()

                for i in list(range(1)):

                    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=depth, min_samples_leaf = leaf),
                                            n_estimators=estimator,loss='linear')

                    regr.fit(x_train, y_train)

                    #make predictions and caculate training FDR
                    PredictTrainY = regr.predict(x_train)

                    PredictTrainY = pd.Series(PredictTrainY)

                    temp = pd.DataFrame(columns = ["PredictTrainY","RealY"])
                    temp["PredictTrainY"] = PredictTrainY
                    temp["RealY"] = y_train
                    temp.sort_values(by = "PredictTrainY", ascending = False, inplace = True)

                    ThreePercent = int(len(x_train) *FDR_Rate)
                    
                    FDR = temp["RealY"][:ThreePercent].sum() / y_train.sum()
                    training_NumberRecords = ThreePercent
                    training_NumberBades = temp["RealY"][:ThreePercent].sum()
                    training_FDR.append(FDR)
                    #print('training FDR' ,FDR)

                    #make predictions and caculate test FDR
                    PredictTestY = regr.predict(x_test)
                    PredictTestY = pd.Series(PredictTestY)
                    PredictTestY

                    temp = pd.DataFrame(columns = ["PredictTestY","RealY"])
                    temp["PredictTestY"] = PredictTestY
                    temp["RealY"] = y_test
                    temp.sort_values(by = "PredictTestY", ascending = False, inplace = True)

                    ThreePercent = int(len(x_test) *FDR_Rate)
                    ThreePercent

                    FDR = temp["RealY"][:ThreePercent].sum() / y_test.sum()
                    testing_NumberRecords = ThreePercent    
                    testing_NumberBades = temp["RealY"][:ThreePercent].sum()
                    testing_FDR.append(FDR)
                    #print('testing FDR' ,FDR)

                    #make predictions and caculate OOT FDR
                    outTimeX = outTime.drop('FraudLabel', axis=1)
                    outTimeY = outTime.loc[:,'FraudLabel']

                    PredictedY = regr.predict(outTimeX)
                    PredictedY = pd.Series(PredictedY)

                    temp = pd.DataFrame(columns = ["PredictedY","RealY"])
                    temp["PredictedY"] = PredictedY
                    temp["RealY"] = outTimeY
                    temp.sort_values(by = "PredictedY", ascending = False, inplace = True)

                    ThreePercent = int(len(outTimeX) *FDR_Rate)
                    ThreePercent

                    FDR = temp["RealY"][:ThreePercent].sum() / outTimeY.sum()
                    OOT_FDR.append(FDR)
                    OOT_NumberRecords = ThreePercent  
                    OOT_NumberBades = temp["RealY"][:ThreePercent].sum()
                    #print('OOT FDR' ,FDR)

                print('max_depth', depth)
                print('n_estimators', estimator)
                print('min_samples_leaf',leaf)
                print('FDR_Rate', FDR_Rate)
                
                print('training_NumberRecords', training_NumberRecords)
                print('training_NumberBades',training_NumberBades)
                print('testing_NumberRecords', testing_NumberRecords)
                print('testing_NumberBades',testing_NumberBades)
                print('OOT_NumberRecords', OOT_NumberRecords)
                print('OOT_NumberBades',OOT_NumberBades)
                
                print('training FDR', np.max(training_FDR))
                print('testing FDR', np.max(testing_FDR))
                print('OOT FDR', np.max(OOT_FDR))
                print('-----------------------------------------')

max_depth 8
n_estimators 100
min_samples_leaf 16
FDR_Rate 0.01
training_NumberRecords 6404
training_NumberBades 4726
testing_NumberRecords 1601
testing_NumberBades 1253
OOT_NumberRecords 1664
OOT_NumberBades 1184
training FDR 0.5141427328111401
testing FDR 0.5331914893617021
OOT FDR 0.49622799664710815
-----------------------------------------
max_depth 8
n_estimators 100
min_samples_leaf 16
FDR_Rate 0.03
training_NumberRecords 19212
training_NumberBades 4857
testing_NumberRecords 4803
testing_NumberBades 1291
OOT_NumberRecords 4994
OOT_NumberBades 1199
training FDR 0.5283942558746736
testing FDR 0.5493617021276596
OOT FDR 0.5025146689019279
-----------------------------------------
max_depth 8
n_estimators 100
min_samples_leaf 16
FDR_Rate 0.05
training_NumberRecords 32021
training_NumberBades 4938
testing_NumberRecords 8005
testing_NumberBades 1329
OOT_NumberRecords 8324
OOT_NumberBades 1230
training FDR 0.5372062663185379
testing FDR 0.565531914893617
OOT FDR 0.5155071248952221
-----

make predictions and caculate training FDR

In [ ]:
'''PredictTrainY = regr.predict(x_train)

PredictTrainY = pd.Series(PredictTrainY)
PredictTrainY.head(5)

temp = pd.DataFrame(columns = ["PredictTrainY","RealY"])
temp["PredictTrainY"] = PredictTrainY
temp["RealY"] = y_train
temp.sort_values(by = "PredictTrainY", ascending = False, inplace = True)

ThreePercent = int(len(x_train) *0.03)

FDR = temp["RealY"][:ThreePercent].sum() / y_train.sum()
FDR'''

make predictions and caculate test FDR

In [ ]:
'''PredictTestY = regr.predict(x_test)
PredictTestY = pd.Series(PredictTestY)
PredictTestY

temp = pd.DataFrame(columns = ["PredictTestY","RealY"])
temp["PredictTestY"] = PredictTestY
temp["RealY"] = y_test
temp.sort_values(by = "PredictTestY", ascending = False, inplace = True)

ThreePercent = int(len(x_test) *0.03)
ThreePercent

FDR = temp["RealY"][:ThreePercent].sum() / y_test.sum()
FDR'''

Seperate OOT data into X and Y

In [ ]:
'''outTimeX = outTime.drop('FraudLabel', axis=1)
outTimeY = outTime.loc[:,'FraudLabel']
inTimeX.head(5)'''


make predictions and caculate OOT FDR

In [ ]:
'''PredictedY = regr.predict(outTimeX)
PredictedY = pd.Series(PredictedY)

temp = pd.DataFrame(columns = ["PredictedY","RealY"])
temp["PredictedY"] = PredictedY
temp["RealY"] = outTimeY
temp.sort_values(by = "PredictedY", ascending = False, inplace = True)

ThreePercent = int(len(outTimeX) *0.03)
ThreePercent

FDR = temp["RealY"][:ThreePercent].sum() / outTimeY.sum()
FDR'''